# NLP 기초대회 미션 2 - 훈련(with.wandb, sweep)



**훈련 미션 : KLUE STS 데이터셋에 다양한 라이브러리를 적용하여 훈련 및 시각화를 진행한다**

**미션 개요**
- Transformer 훈련을 위한 다양한 라이브러리 사용해보기

**실습 배경 및 목적**
- STS 데이터에 알맞는 pytorch lightning 코드 작성
- wandb를 이용한 훈련 결과 시각화
- sweep을 활용한 하이퍼파라미터 최적화

**데이터셋(https://klue-benchmark.com/tasks/67/overview/description)**
- KLUE의 Semantic Textual Similarity(STS) 데이터셋
- 입력 : 두 문장
- 출력 : 두 문장의 유사도
- 학습 데이터 : 11,668개
- 검증 데이터 : 519개(평가 데이터가 비공개이므로 학습에서 평가데이터로 활용함)
- 평가 데이터 : 1,037개(비공개)
- License : <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />

**모델**
- [klue/roberta-small](https://huggingface.co/klue/roberta-small) 모델과 토크나이저 활용

# 환경설정

In [2]:
# !pip install transformers
# !pip install wandb -qU
# !pip install pytorch_lightning

In [3]:
import os
import argparse
import requests

import pandas as pd
import json

from tqdm.auto import tqdm

import transformers
import torch
import torchmetrics
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger

/home/kingstar/anaconda3/envs/ml2/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.996-ko-0.9.2 is an invalid version and will not be supported in a future release
  warnings.warn(


# 데이터 모듈

In [4]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, inputs, targets=[]):
        self.inputs = inputs
        self.targets = targets

    # 학습 및 추론 과정에서 데이터를 1개씩 꺼내오는 곳
    def __getitem__(self, idx):
        # 정답이 있다면 if문을, 없다면 else문을 수행합니다
        if len(self.targets) == 0:
            return torch.tensor(self.inputs[idx])
        else:
            return torch.tensor(self.inputs[idx]), torch.tensor(self.targets[idx])

    # 입력하는 개수만큼 데이터를 사용합니다
    # 'return 100'이면 1에폭에 100개의 데이터만 사용합니다
    def __len__(self):
        return len(self.inputs)

In [5]:
class Dataloader(pl.LightningDataModule):
    def __init__(self, model_name, batch_size, train_ratio, shuffle, bce):
        super().__init__()
        self.model_name = model_name
        self.batch_size = batch_size
        self.train_ratio = train_ratio
        self.shuffle = shuffle
        self.bce = bce

        self.train_dataset = None
        self.val_dataset = None
        self.test_dataset = None
        self.predict_dataset = None

        self.tokenizer = transformers.AutoTokenizer.from_pretrained(model_name, max_length=160)
        self.set_preprocessing()

    def read_json(self, data_type):
        # 파일이 없으면 다운로드 받아옵니다
        if not os.path.isfile(f"{data_type}.json"):
            data = requests.get(f"https://raw.githubusercontent.com/htw5295/klue_sts/main/klue-sts-v1.1_{data_type}.json")
            with open(f"{data_type}.json", 'w', encoding='utf-8') as f:
                json.dump(json.loads(data.text), f, ensure_ascii=False)

        # json 파일을 읽어 pandas 형태로 변환합니다
        with open(f'{data_type}.json', 'r', encoding='utf-8') as file:
            json_data = json.load(file)

        data = []
        for item in json_data:
            data.append([item['source'], item['sentence1'], item['sentence2'], item['labels']['label'], item['labels']['binary-label']])

        df = pd.DataFrame(data, columns=['source', 'sentence1', 'sentence2', 'label', 'binary_label'])

        return df

    def tokenizing(self, dataframe):
        data = []
        for idx, item in tqdm(dataframe.iterrows(), desc='tokenizing', total=len(dataframe)):
            # 두 입력 문장을 [SEP] 토큰으로 이어붙여서 전처리합니다.
            text = '[SEP]'.join([item[text_column] for text_column in self.text_columns])
            outputs = self.tokenizer(text, add_special_tokens=True, padding='max_length', truncation=True)
            data.append(outputs['input_ids'])

        return data

    def set_preprocessing(self):
        # 데이터를 읽어서 타겟 컬럼, 안쓰는 컬럼, 텍스트 전처리가 필요한 컬럼명을 기록합니다.
        data = self.read_json('train')
        columns = data.columns
        if self.bce:
            self.target_columns = [columns[4]]
        else:
            self.target_columns = [columns[3]]

        self.delete_columns = [columns[0]]
        self.text_columns = [columns[1], columns[2]]

    def preprocessing(self, data):
        # 안쓰는 컬럼을 삭제합니다.
        data = data.drop(columns=self.delete_columns)

        # 타겟 데이터가 없으면 빈 배열을 리턴합니다.
        try:
            targets = data[self.target_columns].values.tolist()
        except:
            targets = []
        # 텍스트 데이터를 전처리합니다.
        inputs = self.tokenizing(data)

        return inputs, targets

    def setup(self, stage='fit'):
        if stage == 'fit':
            total_data = self.read_json('train')

            # 학습 데이터와 검증 데이터셋을 비율에 맞춰 분리합니다
            train_data = total_data.sample(frac=self.train_ratio)
            val_data = total_data.drop(train_data.index)

            # 학습데이터 준비
            train_inputs, train_targets = self.preprocessing(train_data)

            # 검증데이터 준비
            val_inputs, val_targets = self.preprocessing(val_data)

            # train 데이터만 shuffle을 적용해줍니다, 필요하다면 val, test 데이터에도 shuffle을 적용할 수 있습니다
            self.train_dataset = Dataset(train_inputs, train_targets)
            self.val_dataset = Dataset(val_inputs, val_targets)
        else:
            # 평가데이터 준비
            test_data = self.read_json('dev')
            test_inputs, test_targets = self.preprocessing(test_data)
            self.test_dataset = Dataset(test_inputs, test_targets)
            self.predict_dataset = Dataset(test_inputs, [])

    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=args.shuffle)

    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return torch.utils.data.DataLoader(self.test_dataset, batch_size=self.batch_size)

    def predict_dataloader(self):
        return torch.utils.data.DataLoader(self.predict_dataset, batch_size=self.batch_size)

# 모델
- pytorch lightning을 활용한 모델 훈련 및 평가 모듈 작성

In [6]:
class Model(pl.LightningModule):
    def __init__(self, model_name, lr, bce):
        super().__init__()
        self.save_hyperparameters()

        self.model_name = model_name
        self.lr = lr
        self.bce = bce

        self.plm = transformers.AutoModelForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_name, num_labels=1)
        if self.bce:
            self.loss_func = torch.nn.BCEWithLogitsLoss()
        else:
            self.loss_func = torch.nn.L1Loss()

    def forward(self, x):
        x = self.plm(x)['logits']

        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_func(logits, y.float())
        self.log("train_loss", loss)

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = self.loss_func(logits, y.float())
        self.log("val_loss", loss)
        if self.bce:
            self.log("val_f1", torchmetrics.functional.f1_score(logits, y, task='binary'))
        else:
            self.log("val_pearson", torchmetrics.functional.pearson_corrcoef(logits, y))

        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        if self.bce:
            self.log("test_f1", torchmetrics.functional.f1_score(logits, y, task='binary'))
        else:
            self.log("test_pearson", torchmetrics.functional.pearson_corrcoef(logits, y, task='binary'))

    def predict_step(self, batch, batch_idx):
        x = batch
        logits = self(x)

        return logits

    def configure_optimizers(self):
        # 이곳에서 lr 값을 변경할 수 있습니다
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr)
        return optimizer

#Train
- wandb를 활용한 모델 훈련 시각화

In [7]:
# 하이퍼 파라미터 등 각종 설정값을 입력받습니다
# 터미널 실행 예시 : python3 run.py --batch_size=64 ...
# 실행 시 '--batch_size=64' 같은 인자를 입력하지 않으면 default 값이 기본으로 실행됩니다
parser = argparse.ArgumentParser()
parser.add_argument('--model_name', default='klue/roberta-small', type=str)
parser.add_argument('--batch_size', default=16, type=int)
parser.add_argument('--max_epoch', default=1, type=int)
parser.add_argument('--shuffle', default=True)
parser.add_argument('--bce', default=True)
parser.add_argument('--train_ratio', default=0.8)
parser.add_argument('--learning_rate', default=1e-5, type=float)
args = parser.parse_args(args=[])

dataloader = Dataloader(args.model_name, args.batch_size, args.train_ratio, args.shuffle, args.bce)
model = Model(args.model_name, args.learning_rate, args.bce)

# wandb logger 설정
wandb_logger = WandbLogger(project="klue-sts")

trainer = pl.Trainer(max_epochs=args.max_epoch, logger=wandb_logger, log_every_n_steps=1)
trainer.fit(model=model, datamodule=dataloader)
trainer.test(model=model, datamodule=dataloader)

# 모델의 predict_step의 결과값을 받아옵니다
predictions = trainer.predict(model=model, datamodule=dataloader)

Some weights of the model checkpoint at klue/roberta-small were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'cla

/home/kingstar/anaconda3/envs/ml2/lib/python3.8/site-packages/pkg_resources/__init__.py:123: PkgResourcesDeprecationWarning: 0.996-ko-0.9.2 is an invalid version and will not be supported in a future release
  warnings.warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


tokenizing:   0%|          | 0/9334 [00:00<?, ?it/s]

tokenizing:   0%|          | 0/2334 [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                             | Params
---------------------------------------------------------------
0 | plm       | RobertaForSequenceClassification | 68.1 M
1 | loss_func | BCEWithLogitsLoss                | 0     
---------------------------------------------------------------
68.1 M    Trainable params
0         Non-trainable params
68.1 M    Total params
272.367   Total estimated model params size (MB)


...

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         test_f1            0.7960061430931091
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


You are using a CUDA device ('NVIDIA GeForce RTX 3060 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


tokenizing:   0%|          | 0/519 [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/kingstar/anaconda3/envs/ml2/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, predict_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Predicting: 0it [00:00, ?it/s]

# Wandb Sweep

- 하이퍼 파라미터 최적화를 위한 도구

In [8]:
# https://docs.wandb.ai/v/ko/sweeps/configuration
# Sweep을 통해 최적화된 hyperparameter를 찾을 수 있습니다.
# 찾기를 원하는 hyperparameter를 다음과 같이 sweep_config에 추가합니다.
# 본 미션에서는 learning rate를 searching하는 예시를 보입니다.

import wandb

sweep_config = {
    'method': 'random', # random: 임의의 값의 parameter 세트를 선택
    'parameters': {
        'lr':{
            'distribution': 'uniform',  # parameter를 설정하는 기준을 선택합니다. uniform은 연속적으로 균등한 값들을 선택합니다.
            'min':1e-5,                 # 최소값을 설정합니다.
            'max':1e-4                  # 최대값을 설정합니다.
        }
    }
}

if args.bce:
    sweep_config['metric'] = {  # sweep_config의 metric은 최적화를 진행할 목표를 설정합니다.
        'name':'val_f1',        # F1 점수가 최대화가 되는 방향으로 학습을 진행합니다.
        'goal':'maximize'
    }
else:
    sweep_config['metric'] = {'name':'val_pearson', 'goal':'maximize'}  # pearson 점수가 최대화가 되는 방향으로 학습을 진행합니다.

In [10]:
# https://docs.wandb.ai/v/ko/sweeps/configuration
# Sweep을 통해 최적화된 hyperparameter를 찾을 수 있습니다.
# 찾기를 원하는 hyperparameter를 다음과 같이 sweep_config에 추가합니다.
# 본 미션에서는 learning rate를 searching하는 예시를 보입니다.

import wandb

sweep_config = {
    'method': 'random', # random: 임의의 값의 parameter 세트를 선택
    'parameters': {
        'lr':{
            'distribution': 'uniform', 'min':1e-5, 'max':1e-4,
        }
    }
}

if args.bce:
    sweep_config['metric'] = {  # sweep_config의 metric은 최적화를 진행할 목표를 설정합니다.
        'name':'val_f1',        # F1 점수가 최대화가 되는 방향으로 학습을 진행합니다.
        'goal':'maximize'
    }
else:
    sweep_config['metric'] = {'name':'val_pearson', 'goal':'maximize'}  # pearson 점수가 최대화가 되는 방향으로 학습을 진행합니다.

In [11]:
# Sweep을 통해 실행될 학습 코드를 작성합니다.

def sweep_train(config=None):
    wandb.init(config=config)
    config = wandb.config

    dataloader = Dataloader(args.model_name, args.batch_size, args.train_ratio, args.shuffle, args.bce)
    model = Model(args.model_name, config.lr, args.bce)
    wandb_logger = WandbLogger(project="klue-sts")

    trainer = pl.Trainer(max_epochs=args.max_epoch, logger=wandb_logger, log_every_n_steps=1)
    trainer.fit(model=model, datamodule=dataloader)
    trainer.test(model=model, datamodule=dataloader)

In [ ]:
# Sweep 생성

sweep_id = wandb.sweep(
    sweep=sweep_config,     # config 딕셔너리를 추가합니다.
    project='klue-sts'  # project의 이름을 추가합니다.
)
wandb.agent(
    sweep_id=sweep_id,      # sweep의 정보를 입력하고
    function=sweep_train,   # train이라는 모델을 학습하는 코드를
    count=3                 # 총 3회 실행해봅니다.
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: 1v7dfut5
Sweep URL: https://wandb.ai/traintogpb/klue-sts/sweeps/1v7dfut5


wandb: Waiting for W&B process to finish... (success).
wandb: | 0.022 MB of 0.022 MB uploaded (0.000 MB deduped)
wandb: Run history:
wandb:               epoch ▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:             test_f1 ▁
wandb:          train_loss ███▇████▇█▇▇██▇▇▄▄▆▂▃▃▃▄▃▄▅▃▃▂▁▂▃▁▃▁▃▃▁▄
wandb: trainer/global_step ▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:              val_f1 ▁
wandb:            val_loss ▁
wandb: 
wandb: Run summary:
wandb:               epoch 1
wandb:             test_f1 0.79601
wandb:          train_loss 0.31497
wandb: trainer/global_step 584
wandb:              val_f1 0.9408
wandb:            val_loss 0.13983
wandb: 
wandb: 🚀 View run crimson-forest-1 at: https://wandb.ai/traintogpb/klue-sts/runs/hdgf9oas
wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
wandb: Find logs at: ./wandb/run-20230412_124255-hdgf9oas/logs
wandb: Agent Starting Run: d8thiwt4 with config:
wandb: 	lr: 7.786021356526342e-05


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Some weights of the model checkpoint at klue/roberta-small were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-small and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'cla

...

tokenizing:   0%|          | 0/2334 [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                             | Params
---------------------------------------------------------------
0 | plm       | RobertaForSequenceClassification | 68.1 M
1 | loss_func | BCEWithLogitsLoss                | 0     
---------------------------------------------------------------
68.1 M    Trainable params
0         Non-trainable params
68.1 M    Total params
272.367   Total estimated model params size (MB)
wandb: WARNING Config item 'lr' was locked by 'sweep' (ignored update).


Sanity Checking: 0it [00:00, ?it/s]

/home/kingstar/anaconda3/envs/ml2/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/kingstar/anaconda3/envs/ml2/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

###**콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다.